<a href="https://colab.research.google.com/github/tenorio-vivianesgm/neuromatch-2020/blob/group-project/Infering_Motor_Intent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Information
`project proposal`:
https://docs.google.com/document/d/1vX-ZZseIFAHCuJs9PbTi6GtOGhWsJlzx8OSeq-VhFVY/edit#

`paper location`: https://www.nature.com/articles/s41586-019-1787-x <br>
`Data Loading Notebook`: https://colab.research.google.com/github/NeuromatchAcademy/course-content/blob/master/projects/load_steinmetz_decisions.ipynb#scrollTo=mmOarX5w16CR <br>
`NMA Project Information`: https://github.com/NeuromatchAcademy/course-content/tree/master/projects



# Initial Dataset Loading and Setup

## Imports and Init Code


In [2]:
import os, requests
import datetime as dt
import sklearn as sk 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#@title Figure Settings
import ipywidgets as widgets
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#@title import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True
print("done")

print(dt.datetime.today().strftime("%I:%M:%S, %A %d, %B %Y"))


done
04:24:33, Wednesday 29, July 2020


### ?debug


In [3]:
debug = False

## Data Retrieval and Loading Code

In [4]:
debug = True if debug is None else debug

if debug:
  !wget 'https://helixcarinae.s3.us-east-1.amazonaws.com/sm_data.npy'
  #!wget 'https://github.com/HelixCarinae/nma_decoding_motion/raw/master/sm_data.npy'
  !ls ./
  data = np.load('./sm_data.npy', allow_pickle=True)
  print(data[0].keys())
  print("Subset of Steinmetz Data for first 4 sessions loaded")

else:
  fname = []
  for j in range(3):
    fname.append('steinmetz_part%d.npz'%j)
  url = ["https://osf.io/agvxh/download"]
  url.append("https://osf.io/uv3mw/download")
  url.append("https://osf.io/ehmw2/download")

  for j in range(len(url)):
    if not os.path.isfile(fname[j]):
      try:
        r = requests.get(url[j])
      except requests.ConnectionError:
        print("!!! Failed to download data !!!")
      else:
        if r.status_code != requests.codes.ok:
          print("!!! Failed to download data !!!")
        else:
          with open(fname[j], "wb") as fid:
            fid.write(r.content)
  
  data = np.array([])
  for j in range(len(fname)):
    data = np.hstack((data, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))
  print("Full Steinmetz dataset loaded")

dat = data[2]



Full Steinmetz dataset loaded


# Data Information and Structure


### dataset information
`alldat` contains 39 sessions from 10 mice, data from Steinmetz et al, 2019. Time bins for all measurements are 10ms, starting 500ms before stimulus onset. The mouse had to determine which side has the highest contrast. For each `dat = alldat[k]`, you have the following fields:

* `dat['mouse_name']`: mouse name
* `dat['date_exp']`: when a session was performed
* `dat['spks']`: neurons by trials by time bins.
* `dat['brain_area']`: brain area for each neuron recorded. 
* `dat['contrast_right']`: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
* `dat['contrast_left']`: contrast level for left stimulus. 
* `dat['gocue']`: when the go cue sound was played. 
* `dat['response_times']`: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.  
* `dat['response']`: which side the response was (`-1`, `0`, `1`). When the right-side stimulus had higher contrast, the correct choice was `-1`. `0` is a no go response. 
* `dat['feedback_time']`: when feedback was provided. 
* `dat['feedback_type']`: if the feedback was positive (`+1`, reward) or negative (`-1`, white noise burst).  
* `dat['wheel']`: exact position of the wheel that the mice uses to make a response, binned at `10ms`. 
* `dat['pupil']`: pupil area  (noisy, because pupil is very small) + pupil horizontal and vertical position. 
* `dat['lfp']`: recording of the local field potential in each brain area from this experiment, binned at `10ms`.
* `dat['brain_area_lfp']`: brain area names for the LFP channels. 
* `dat['trough_to_peak']`: measures the width of the action potential waveform for each neuron. Widths `<=10` samples are "putative fast spiking neurons". 
* `dat['waveform_w']`: temporal components of spike waveforms. `w@u` reconstructs the time by channels action potential shape. 
* `dat['waveform_u]`: spatial components of spike waveforms.
* `dat['%X%_passive']`: same as above for `X` = {`spks`, `lfp`, `pupil`, `wheel`, `contrast_left`, `contrast_right`} but for  passive trials at the end of the recording when the mouse was no longer engaged and stopped making responses. 

### (Potentially) Useful Data
* `dat['spks']`: neurons by trials by time bins.
* `dat['brain_area']`: brain area for each neuron recorded. 
* `dat['gocue']`: when the go cue sound was played. 
* `dat['response_times']`: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.
* `dat['response']`: which side the response was (`-1`, `0`, `1`). When the right-side stimulus had higher contrast, the correct choice was `-1`. `0` is a no go response. 
* `dat['feedback_time']`: when feedback was provided. 
* `dat['feedback_type']`: if the feedback was positive (`+1`, reward) or negative (`-1`, white noise burst). 






In [5]:
#@title Structure Information

print((data.shape))
print((data[0].keys()))
print()

print(type(data[0]))
print(len(data[0]), '\n')

print(type(data[0]['spks']))
print(len(data[0]['spks']), '\n')
print((data[0]['spks'][:1][:10][0][0][0]))

print()
print(type(data[0]['spks'][0]))
print(len(data[0]['spks'][0]), '\n')

print(type(data[0]['spks'][0][0]))
print(len(data[0]['spks'][0][0]), '\n')


## data structure
# NDARRY of dictionaries containing N ndarrays
#print(data[0]['spks'])

(39,)
dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])

<class 'dict'>
31 

<class 'numpy.ndarray'>
734 

0

<class 'numpy.ndarray'>
214 

<class 'numpy.ndarray'>
250 



#Initial Analysis



In [75]:
#@title Select Brain Areas


df = pd.Series(np.nan, index=['MOs'])


for k in range(len(data)):  
  aux = pd.Series(np.where(data[k]['brain_area'] == 'MOs'))
  df = df.append(aux, ignore_index=True)

print(df)

NN = len(dat['brain_area']) # number of neurons


0                                                   NaN
1     [1, 4, 7, 8, 12, 17, 19, 22, 23, 25, 26, 27, 2...
2                                                    []
3                                                    []
4     [947, 954, 958, 971, 979, 982, 983, 985, 996, ...
5     [786, 810, 814, 822, 831, 845, 847, 851, 878, ...
6                                                    []
7                                                    []
8     [3, 14, 16, 20, 23, 24, 25, 26, 39, 46, 48, 57...
9                                                    []
10                                                   []
11                                                   []
12                       [227, 297, 322, 329, 330, 393]
13    [518, 520, 534, 555, 559, 563, 567, 591, 593, ...
14    [1, 3, 6, 7, 8, 9, 11, 12, 14, 15, 16, 17, 18,...
15                                                   []
16                                                   []
17                                              

In [ ]:
#@title Select Active Neurons

# We need to get the contents of the previous variables (mos, mop, ssp, olf) 
# from 39 sessions and use them as indexes to data['spks'] to select the active 
# neurons in each region

# E.G: active_neuron [session 1][mop[0]] -> meaning neuron 1 was active in section 1
print(mos)


(array([ 845,  875,  880,  883,  895,  904,  919,  921,  950,  995, 1003,
       1010, 1028, 1039, 1041, 1046, 1054, 1057, 1060, 1067, 1072, 1076,
       1078, 1080, 1085, 1088, 1093, 1100, 1101, 1108, 1113, 1131, 1139,
       1156, 1163, 1165, 1186, 1202, 1242, 1260, 1292, 1293, 1296, 1300,
       1303, 1305, 1320, 1324, 1328, 1332, 1334, 1335, 1338, 1339]),)


In [ ]:
plt.xlim(50,75)
plt.legend();#Plot 2: Plot 2: Response Times x Response

#we want to observe at which points the mice may have moved the wheel before the go cue
#plot the left, right, and no-go responses compared to the response time
#variables: dat['response_times']: when the response is registered
            #dat['response']: which side the response was on [-1, 0, 1]--left, no go, right  

from matplotlib import pyplot as plt
response = dat['response'] # right - nogo - left (-1, 0, 1)
dt = dat['bin_size']
plt.plot()


In [ ]:
#@title Input Matrix
########################################################################
#                 INPUT MATRIX
# In our case, our input is the stimulus (contrast) 
# and our output is the movement (response).
# Since we dont care about constrast level initially,
# we can separate the data as follows:
########################################################################

#Create local variables
contrast_left = data[0]['contrast_left']
contrast_right = data[0]['contrast_right']
feedback = data[0]['feedback_type']

# Initialize arrays
expected_right = np.zeros(len(contrast_right))
expected_left = np.zeros(len(contrast_left))

# Find expected answer(movement to the right)
for i in range(len(contrast_right)):
  expected_right[i] = contrast_right[i] > contrast_left[i]
  expected_left[i] = contrast_left[i] > contrast_right[i]

expected_right = expected_right*(-1)

input = expected_left+expected_right #original data
print(input)

In [ ]:
#@title Output Matrix

# Our output is simply the response from the mice.

output = dat['response']
print(output)

In [ ]:
#@title Resample with Replacement

def resample_with_replacement(x, y):
  """Resample data points with replacement from the dataset of `x` inputs and
  `y` measurements.
  Args:
    x (ndarray): An array of shape (samples,) that contains the input values.
    y (ndarray): An array of shape (samples,) that contains the corresponding
      measurement values to the inputs.
  Returns:
    ndarray, ndarray: The newly resampled `x` and `y` data points.
  """

  # Get array of indices for resampled points
  sample_idx = np.random.choice(len(x), size=len(x), replace=True)

  # Sample from x and y according to sample_idx
  x_ = x[sample_idx]
  y_ = y[sample_idx]

  return x_, y_

#input_ -> training data

input_, output_ = resample_with_replacement(input, output)
print(input_)

In [ ]:
#@title Fitting the model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_reg = LogisticRegression(penalty="none")
input_ = input_.reshape(-1,1)
log_reg.fit(input_, output_)
output_pred = log_reg.predict(input_)

In [ ]:
#@title Accuracy

def compute_accuracy(X, y, model):
  """Compute accuracy of classifier predictions.
  
  Args:
    X (2D array): Data matrix
    y (1D array): Label vector
    model (sklearn estimator): Classifier with trained weights.
  Returns:
    accuracy (float): Proportion of correct predictions.  
  """
  y_pred = model.predict(X)
  accuracy = (y == y_pred).mean()

  return accuracy
input = input.reshape(-1,1)
train_accuracy = compute_accuracy(input, output, log_reg)
print(f"Accuracy on the training data: {train_accuracy:.2%}")

In [ ]:
#@title Cross-validation

accuracies = cross_val_score(LogisticRegression(penalty='none'), input, output, cv=8)

In [ ]:
#@title
#@markdown Run to plot out these `k=8` accuracy scores.
f, ax = plt.subplots(figsize=(8, 3))
ax.boxplot(accuracies, vert=False, widths=.7)
ax.scatter(accuracies, np.ones(8))
ax.set(
  xlabel="Accuracy",
  yticks=[],
  title=f"Average test accuracy: {accuracies.mean():.2%}"
)
ax.spines["left"].set_visible(False)